In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_cleaning import load_trade
from refreshTime import refresh_time, test_date, resample,synchro_data,harmoniz_data
from Clustering import get_clusters
from GMVP import get_GMVP,get_Louvain_GMVP

In [2]:
Date = pd.bdate_range('2010-01-01','2010-12-31')
Market_name = np.array(['AAPL.OQ','AMGN.OQ','AXP.N','BA.N','CAT.N','CSCO.OQ','CVX.N','DOW.N','GS.N','SPY.P','UTX.N','V.N','WMT.N'])

#folder_path = 'D:/GitHub/ML_For_Finance/big data/data/data/'

folder_path ='D:/GitHub/ML for Finance project/ML_For_Finance/big data/Data/data/'
#folder_path = '/Users/franckatteaka/Desktop/cours/Semester III/Financial big data/high freq data/'

In [3]:
aapl = load_trade(Market_name[0],Date[7],folder_path,is_compressed = True)
amgn = load_trade(Market_name[1],Date[7],folder_path,is_compressed = True)
axpn = load_trade(Market_name[2],Date[7],folder_path,is_compressed = True)

In [4]:
test1= [aapl.iloc[:1000],amgn.iloc[:1000],axpn.iloc[:1000]]

In [6]:
test2=[]
for i in range(len(Market_name)):
    data0=load_trade(Market_name[i],Date[7],folder_path,is_compressed = True)
    test2.append(data0)

In [ ]:
data1=harmoniz_data(test2)

D:\GitHub\ML for Finance project\ML_For_Finance\big data\Code\refreshTime.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dates = np.array([df.index.drop_duplicates(keep = 'last') for df in dfs])
D:\GitHub\ML for Finance project\ML_For_Finance\big data\Code\refreshTime.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dates = np.array([date[np.where(date > tau[-1])[0]] for date in dates ])


In [25]:
C = data1.corr()

In [27]:
q=len(test2)/len(data1.index)
get_clusters(C,q)

array([1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
get_Louvain_GMVP(data1)

In [32]:
q

0.1511627906976744

In [11]:
w,mu,sig=get_GMVP(data1)

In [13]:
data1.values@w

array([51.96308147, 51.95491625, 51.93055401, 51.9162836 , 51.95368577,
       51.95543528, 51.95533785, 51.98825103, 51.98663856, 51.96178249,
       51.90664012, 51.92260161, 51.95111438, 51.95541257, 51.9483127 ,
       51.95246676, 51.96163811, 51.94785961, 51.96647958, 51.97320208,
       51.9893525 , 51.98667358, 52.00370247, 51.99409156, 51.98880108,
       51.98242483, 51.9851686 , 51.98874274, 51.98101922, 51.97884653,
       51.97924405, 51.98466612, 51.97564698, 51.98908157, 51.98753431,
       51.99637633, 52.00850896, 51.99939274, 51.9874117 , 51.96843513,
       51.94064053, 51.94665264, 51.9728856 , 51.97172266, 51.99113447,
       51.99521408, 52.00134836, 51.99421349, 51.97179385, 51.97021137,
       51.99044751, 52.01356295, 52.01051944, 52.01439479, 52.0018604 ,
       51.99602941, 51.98383266, 51.98712432, 52.01033361, 52.03100565,
       52.03782776, 52.02570601, 52.02687489, 52.02027856, 52.00964433,
       51.94762615, 51.95141739, 51.9422244 , 51.94389239, 51.95

In [6]:
return_data = data1- data1.shift(1)
return_data=return_data.dropna()
np.mean(return_data,axis=0)

AAPL.OQ   -0.001983
AMGN.OQ    0.001941
AXP.N      0.003502
BA.N       0.002278
CAT.N     -0.000211
CSCO.OQ   -0.000464
CVX.N      0.000506
DOW.N      0.001603
GS.N      -0.006203
SPY.P      0.000211
UTX.N      0.000295
V.N       -0.000675
WMT.N      0.001097
dtype: float64

In [7]:
return_data

,AAPL.OQ,AMGN.OQ,AXP.N,BA.N,CAT.N,CSCO.OQ,CVX.N,DOW.N,GS.N,SPY.P,UTX.N,V.N,WMT.N
2010-01-12 09:32:09.097007999-05:00,0.23,0.16,0.02,-0.01,-0.08,0.00,-0.07,0.00,-0.14,0.010,-0.01,0.00,0.03
2010-01-12 09:32:20.536972800-05:00,0.15,-0.09,0.00,-0.09,0.00,0.00,0.02,0.01,-0.05,0.005,-0.06,0.00,0.03
2010-01-12 09:32:41.883993600-05:00,-0.03,-0.02,-0.05,-0.05,-0.03,-0.01,0.03,-0.02,-0.04,-0.005,0.03,-0.04,0.00
2010-01-12 09:33:13.200969600-05:00,-0.10,0.04,0.08,0.13,-0.09,-0.03,0.03,0.10,0.31,0.070,-0.03,0.02,-0.02
2010-01-12 09:33:19.067011200-05:00,-0.01,0.02,0.03,0.00,-0.03,0.00,-0.01,-0.03,-0.02,0.010,-0.01,0.05,-0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-12 10:21:19.486972800-05:00,-0.22,-0.01,-0.01,-0.03,-0.08,0.00,-0.05,-0.01,-0.03,-0.050,-0.03,-0.01,-0.03
2010-01-12 10:21:27.502041600-05:00,-0.02,-0.02,-0.02,-0.02,-0.01,-0.02,0.00,-0.04,0.02,-0.020,-0.01,-0.01,-0.02
2010-01-12 10:21:32.397984-05:00,0.00,-0.01,0.00,-0.01,-0.04,-0.01,-0.01,-0.04,-0.02,-0.030,-0.02,-0.01,-0.01
2010-01-12 10:21:59.190019200-05:00,0.09,0.01,0.03,-0.01,0.04,0.01,0.04,0.01,0.07,0.020,0.01,-0.02,0.02


In [8]:
data1

,AAPL.OQ,AMGN.OQ,AXP.N,BA.N,CAT.N,CSCO.OQ,CVX.N,DOW.N,GS.N,SPY.P,UTX.N,V.N,WMT.N
2010-01-12 09:31:51.134966400-05:00,208.81,56.81,41.36,60.21,62.80,24.46,79.79,30.43,169.70,113.920,71.59,86.50,53.94
2010-01-12 09:32:09.097007999-05:00,209.04,56.97,41.38,60.20,62.72,24.46,79.72,30.43,169.56,113.930,71.58,86.50,53.97
2010-01-12 09:32:20.536972800-05:00,209.19,56.88,41.38,60.11,62.72,24.46,79.74,30.44,169.51,113.935,71.52,86.50,54.00
2010-01-12 09:32:41.883993600-05:00,209.16,56.86,41.33,60.06,62.69,24.45,79.77,30.42,169.47,113.930,71.55,86.46,54.00
2010-01-12 09:33:13.200969600-05:00,209.06,56.90,41.41,60.19,62.60,24.42,79.80,30.52,169.78,114.000,71.52,86.48,53.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-12 10:21:19.486972800-05:00,208.08,57.29,42.14,60.79,62.75,24.35,79.87,30.87,168.12,113.990,71.68,86.34,54.20
2010-01-12 10:21:27.502041600-05:00,208.06,57.27,42.12,60.77,62.74,24.33,79.87,30.83,168.14,113.970,71.67,86.33,54.18
2010-01-12 10:21:32.397984-05:00,208.06,57.26,42.12,60.76,62.70,24.32,79.86,30.79,168.12,113.940,71.65,86.32,54.17
2010-01-12 10:21:59.190019200-05:00,208.15,57.27,42.15,60.75,62.74,24.33,79.90,30.80,168.19,113.960,71.66,86.30,54.19
